In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
data = pd.read_csv('./ukiyo-e_crawler/data/ukiyo-e_artworks_details.csv')
data.shape

(177985, 8)

In [3]:
print(data.columns)

Index(['Image URL', 'Artist', 'Title', 'Date', 'Details', 'Source',
       'Description', 'Similar Prints'],
      dtype='object')


In [4]:
# get the number of non-null values in description column
data['Description'].notnull().sum()

82648

In [21]:
# find entries with a 'clock' in the description
data['Description'].str.contains('').sum()
# 

149

In [22]:
data[data['Description'].str.contains('clock')]['Description']

ValueError: Cannot mask with non-boolean array containing NA / NaN values